In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# !pip install cupy-cuda11x>=12.0.0
# !pip install packaging>=22
# !pip install shapely>=2.0.1
# !pip install numpy<1.26
# !pip install scipy<1.12
!pip install transformers datasets accelerate peft


In [ ]:
# You only need to run this once per machine
!pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q -U datasets scipy ipywidgets
!pip install datasets

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset('csv', data_files='/kaggle/input/lmsys-chatbot-arena/test.csv')

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config,device_map="auto")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
train_data = pd.read_csv('../input/lmsys-chatbot-arena/train.csv')
test_data = pd.read_csv('../input/lmsys-chatbot-arena/test.csv')
submission_data = pd.read_csv('../input/lmsys-chatbot-arena/sample_submission.csv')

In [ ]:
train_data.shape,test_data.shape,submission_data.shape

def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

def trim_endings(custom_string):
    return custom_string[:-2][2:]

def count_newlines(custom_string):
    return custom_string.count('\\n')

def word_counts(custom_string):
    return len(custom_string.split())


def apply_transformations(df):
    
    df['prompt'] =  df['prompt'].map(process)
    df['response_a'] =  df['response_a'].map(process)
    df['response_b'] =  df['response_b'].map(process)
    
    df['prompt'] =  df['prompt'].map(trim_endings)
    df['response_a'] =  df['response_a'].map(trim_endings)
    df['response_b'] =  df['response_b'].map(trim_endings)
    
    df['res_a_line_count'] = df['response_a'].map(count_newlines).astype(str)
    df['res_b_line_count'] = df['response_b'].map(count_newlines).astype(str)
    
    df['res_a_word_count'] = df['response_a'].map(word_counts).astype(str)
    df['res_b_word_count'] = df['response_b'].map(word_counts).astype(str)
    
    return df

def create_summary(df):
    df['summary'] = 'User prompt: ' + df['prompt'] +  '\n\n Model A :\n' + df['response_a'] +'\n\n Model A length:\n' + df['res_a_word_count'] +'\n\n Model A Line Count:\n' + df['res_a_line_count'] +'\n\nModel B:\n'  + df['response_b'] +'\n\n \n\nModel B length:\n'  + df['res_b_word_count'] +'\n\n \n\nModel B Line Count:\n'  + df['res_b_line_count']
    return df

In [ ]:
train_data = apply_transformations(train_data)
train_data = create_summary(train_data)
test_data = apply_transformations(test_data)    
test_data = create_summary(test_data)

In [ ]:
train_data['summary'][2]

In [ ]:
train_data['labels'] = train_data[['winner_model_a','winner_model_b','winner_tie']].idxmax(axis=1)
train_data['labels']=train_data['label'].astype('category')
train_data['target']=train_data['label'].cat.codes

In [ ]:
category_map = {code: category for code, category in enumerate(train_data['labels'].cat.categories)}
category_map

In [ ]:
SEED = 7920

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, classification_report, balanced_accuracy_score, accuracy_score
train_df = train_data[['id','summary','target']]
test_df = train_data[['id','summary']]

df_train,df_val = train_test_split(train_df,test_size=0.2,random_state=SEED,stratify=train_df['target'])

In [ ]:
from datasets import Dataset, DatasetDict
dataset_train = Dataset.from_pandas(df_train,preserve_index=False)
dataset_val = Dataset.from_pandas(df_val,preserve_index=False)
dataset_test = Dataset.from_pandas(test_df,preserve_index=False)

In [ ]:
 dataset_train_shuffled = dataset_train.shuffle(seed=SEED)

In [ ]:
dataset = DatasetDict({
    'train': dataset_train_shuffled,
#     'val': dataset_val,
#     'test': dataset_test
})
dataset

In [ ]:
df_train.target.value_counts(normalize=True)

In [ ]:
class_weights=(1/df_train.target.value_counts(normalize=True).sort_index()).tolist()
class_weights=torch.tensor(class_weights)
class_weights=class_weights/class_weights.sum()
class_weights

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=512,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

In [ ]:
MAX_LEN = 512
col_to_delete = ['summary']

def llama_preprocessing_function(examples):
    return tokenizer(examples['summary'], truncation=True, max_length=MAX_LEN)

tokenized_datasets = dataset.map(llama_preprocessing_function, batched=True, remove_columns=col_to_delete)
tokenized_datasets = tokenized_datasets.rename_column("target", "labels")
tokenized_datasets.set_format("torch")

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [ ]:
print(model)

In [ ]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

# Apply the accelerator. You can comment this out to remove the accelerator.
model = accelerator.prepare_model(model)

In [ ]:
!pip install -q wandb -U

import wandb, os
wandb.login()

wandb_project = "mistral-finetune"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project

In [ ]:
if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

In [ ]:
!nvcc --version
!pip uninstall torch torchvision torchaudio -y
!pip install torch==2.0.0+cu121 torchvision==0.15.0+cu121 torchaudio==2.0.0+cu121 -f https://download.pytorch.org/whl/nightly/cu121/torch_nightly.html


In [ ]:
# Uninstall existing versions of torch, torchvision, and torchaudio
!pip uninstall torch torchvision torchaudio -y

# Install compatible versions with CUDA 11.2 support
!pip install torch torchvision torchaudio -f https://download.pytorch.org/whl/cu112/torch_stable.html


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'


In [ ]:
import torch

seed = 7920  # Replace with your desired seed

if torch.cuda.is_available():
    generator = torch.Generator(device='cuda').manual_seed(seed)
else:
    generator = torch.Generator().manual_seed(seed)


In [ ]:
if torch.cuda.is_available():
  generator = torch.Generator('cuda').manual_seed(seed)
else:
  generator = torch.Generator().manual_seed(seed)

In [ ]:
import transformers
class CustomTrainer(transformers.Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Ensure label_weights is a tensor
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32).to(self.args.device)
        else:
            self.class_weights = None

    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract labels and convert them to long type for cross_entropy
        labels = inputs.pop("labels").long()

        # Forward pass
        outputs = model(**inputs)

        # Extract logits assuming they are directly outputted by the model
        logits = outputs.get('logits')

        # Compute custom loss with class weights for imbalanced data handling
        if self.class_weights is not None:
            loss = F.cross_entropy(logits, labels, weight=self.class_weights)
        else:
            loss = F.cross_entropy(logits, labels)

        return (loss, outputs) if return_outputs else loss

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'balanced_accuracy' : balanced_accuracy_score(predictions, labels),'accuracy':accuracy_score(predictions,labels)}

In [ ]:
training_args = transformers.TrainingArguments(
    output_dir = 'model_classification',
    learning_rate = 1e-4,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 1,
    weight_decay = 0.01,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True,
	logging_steps=10,
	max_steps=100,
    fp16=True,
    push_to_hub=False
)

In [ ]:
collate_fn = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_datasets['train'],
    tokenizer = tokenizer,
    data_collator = collate_fn,
    compute_metrics = compute_metrics,
    class_weights=class_weights,
)

In [ ]:
!pip install torch

In [ ]:
import transformers
from datetime import datetime
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Check if CUDA is available
print("CUDA Available:", torch.cuda.is_available())
project = "mistral-finetune"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_datasets['train'],
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=2,
        gradient_checkpointing=True,
        gradient_accumulation_steps=4,
        max_steps=1000,
        learning_rate=2.5e-5, # Want about 10x smaller than the Mistral learning rate
        logging_steps=50,
        fp16=False,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=50,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=50,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
        report_to="wandb",           # Comment this out if you don't want to use weights & baises
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run (optional)
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
    trust_remote_code=True,
)

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "mistral-viggo-finetune/checkpoint-1000")

In [ ]:
eval_prompt = """Given a target sentence construct the underlying meaning representation of the input sentence as a single function with attributes and attribute values.
This function should describe the target string accurately and the function must be one of the following ['inform', 'request', 'give_opinion', 'confirm', 'verify_attribute', 'suggest', 'request_explanation', 'recommend', 'request_attribute'].
The attributes must be one of the following: ['name', 'exp_release_date', 'release_year', 'developer', 'esrb', 'rating', 'genres', 'player_perspective', 'has_multiplayer', 'platforms', 'available_on_steam', 'has_linux_release', 'has_mac_release', 'specifier']

### Target sentence:
Earlier, you stated that you didn't have strong feelings about PlayStation's Little Big Adventure. Is your opinion true for all games which don't have multiplayer?

### Meaning representation:
"""

model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))